In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
all_drinks = pd.read_pickle('./ing_meas_processed_alcohol.pkl')
compared_alcohol = pd.read_pickle('./preprocessed_alcohol.pkl')

#### 문제점

L 단위의 경우 l 한 글자여서, 필터에서 문자들의 l을 가져와 버리는 문제가 있다.
해당 글자를 기존 데이터와 비교하여서 처리할 필요가 있다.

이는 gr도 마찬가지이다. 단위가 2개 이상 포함되는 경우 뒤의 단위를 삭제하는 알고리즘이 필요하다.

(ex) 'oz', 'l' 이렇게 검색해 보면 'l'이 chilled와 같이 단어에 l이 들어간 경우로 매칭된 것을 알 수 있다.

In [16]:
text = "4 tablespoons\r\n"
result = re.sub("\\r|\\n", "", text)
result

'4 tablespoons'

In [3]:
# verify measure preprocess   # remeber index 74, 85, 103, 163 ...
for i in range(len(all_drinks)):
    print("index: ", i)
    print(all_drinks['Measures'][i])
    print(compared_alcohol['Measures'][i])
    print()

index:  0
[[['1', 'oz'], 1], [['1', 'oz'], 2]]
[['1 oz white ', 1], ['1 oz ', 2]]

index:  1
[[['1/2', 'oz'], 1], [['1/4', 'oz'], 2], [['1/4', 'oz'], 3], [['1/4', 'oz'], 4], [['1/4', 'oz'], 5], [['1/4', 'oz'], 6], [['1/2', 'oz'], 7], [['1/4', 'oz'], 8]]
[['1/2 oz ', 1], ['1/4 oz ', 2], ['1/4 oz ', 3], ['1/4 oz ', 4], ['1/4 oz ', 5], ['1/4 oz ', 6], ['1/2 oz ', 7], ['1/4 oz ', 8]]

index:  2
[[['16', 'oz'], 1], [['1.5', 'oz'], 2]]
[['16 oz ', 1], ['1.5 oz ', 2]]

index:  3
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1', 'oz'], 4], [['1', 'oz'], 5], [['3', 'oz'], 6], [['1', 'oz'], 7], [['1', 'cup'], 8]]
[['1/2 oz ', 1], ['1/2 oz ', 2], ['1/2 oz Bacardi ', 3], ['1 oz ', 4], ['1 oz ', 5], ['3 oz ', 6], ['1 oz ', 7], ['1 cup ', 8]]

index:  4
[[['1', 'shot'], 1], [['2', 'shot'], 2], [['1', 'shot'], 3], [['1', 'shot'], 4]]
[['1 shot ', 1], ['2 shots ', 2], ['1 shot ', 3], ['1 shot ', 4]]

index:  5
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1/2', 'oz'

In [4]:
UNIT = ["oz", "ounce", "fl oz", "cl", "tsp", "cubes", "tblsp", "tablespoon" "dash", "dashes", "cup", "pinch", "shot", "glass", "glasses", "drop",
        "can", "jigger", "dl", "bottle", "pint", "quart", "qt", "gr", "inch", "splash", "gal", "scoop", "scoops",
        "kg", "lb", "ml", "part", "top up with", "fill to top", "fill", "to fill", "a little bit of", "slice", "juice of",
        "sprig", "sprigs", "wedge", "twist of", "long strip", "whole", "large", "fifth", "piece", "pieces", "chunk", "by taste", "to taste",
        "handful", "stick", "package", "packages", "leaves", "l"]

In [5]:
# only one unit will survive
def remove_duplicate_unit(list_):
    for i in range(len(list_)):
        element = list_[i][0]
        count = 0
        remove_index = []
        for index in range(len(element)):
            if element[index] in UNIT:
                count+=1
            if count >= 2:
                remove_index.append(index)
        for del_index in reversed(remove_index):
            element.pop(del_index)
    return list_

all_drinks['Measures'] = all_drinks['Measures'].apply(remove_duplicate_unit)

In [6]:
# verify measure preprocess   # remeber index 74, 85, 103, 163 ...
for i in range(len(all_drinks)):
    print("index: ", i)
    print(all_drinks['Measures'][i])
    print(compared_alcohol['Measures'][i])
    print()

index:  0
[[['1', 'oz'], 1], [['1', 'oz'], 2]]
[['1 oz white ', 1], ['1 oz ', 2]]

index:  1
[[['1/2', 'oz'], 1], [['1/4', 'oz'], 2], [['1/4', 'oz'], 3], [['1/4', 'oz'], 4], [['1/4', 'oz'], 5], [['1/4', 'oz'], 6], [['1/2', 'oz'], 7], [['1/4', 'oz'], 8]]
[['1/2 oz ', 1], ['1/4 oz ', 2], ['1/4 oz ', 3], ['1/4 oz ', 4], ['1/4 oz ', 5], ['1/4 oz ', 6], ['1/2 oz ', 7], ['1/4 oz ', 8]]

index:  2
[[['16', 'oz'], 1], [['1.5', 'oz'], 2]]
[['16 oz ', 1], ['1.5 oz ', 2]]

index:  3
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1', 'oz'], 4], [['1', 'oz'], 5], [['3', 'oz'], 6], [['1', 'oz'], 7], [['1', 'cup'], 8]]
[['1/2 oz ', 1], ['1/2 oz ', 2], ['1/2 oz Bacardi ', 3], ['1 oz ', 4], ['1 oz ', 5], ['3 oz ', 6], ['1 oz ', 7], ['1 cup ', 8]]

index:  4
[[['1', 'shot'], 1], [['2', 'shot'], 2], [['1', 'shot'], 3], [['1', 'shot'], 4]]
[['1 shot ', 1], ['2 shots ', 2], ['1 shot ', 3], ['1 shot ', 4]]

index:  5
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1/2', 'oz'

In [7]:
# create regex filter
filter = ""
for unit in UNIT:
    if (unit == "oz"):
        filter += unit
    else:
        filter += "|"+ unit
filter

'oz|ounce|fl oz|cl|tsp|cubes|tblsp|tablespoondash|dashes|cup|pinch|shot|glass|glasses|drop|can|jigger|dl|bottle|pint|quart|qt|gr|inch|splash|gal|scoop|scoops|kg|lb|ml|part|top up with|fill to top|fill|to fill|a little bit of|slice|juice of|sprig|sprigs|wedge|twist of|long strip|whole|large|fifth|piece|pieces|chunk|by taste|to taste|handful|stick|package|packages|leaves|l'

#### Create Rule-Based Algorithm

Rule-Based 기반의 알고리즘을 통해, 단위만 분류될 수 있도록 합니다.

여기서 생성된 vis는 아래의 추가적인 전처리 함수를 위해 사용될 것입니다.

vis 결과를 추가적으로 전처리해도, 좋은 결과를 만들 수 있습니다. 그러나 juice of 처럼 구 단위로 되어 있는 경우

띄어쓰기 구분에서 의하여 각각 따로 분류 되기 때문에, 추가적인 규칙을 정의하여야 합니다.

In [8]:
number = ['1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '/']  # empty space or / for the case of number/unit differentiation
character = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

vis = []
for i in range(len(compared_alcohol)):
    vis.append([])
    for j in range(len(compared_alcohol['Measures'][i])):
        text = str(compared_alcohol['Measures'][i][j][0]).lower() # lower cased
        text = re.sub(',', '', text)
        text = re.sub('\\r|\\n', '', text)

        unit_index = re.search(filter, text) # index_start ~ index_end-1 까지가 포함되는 단위이다. 즉 [start_index : end_index] 이렇게 사용할 수 있다.
        update = True


        if unit_index: # in case of not NoneType -> matched at least one
            start_index, end_index = unit_index.start(), unit_index.end()
            if (start_index == 0) and (end_index == len(text)): # 단위만 나오는 경우
                pass
            elif (start_index != 0) and (end_index == len(text)): # 패턴에 끝에 매칭되는 경우
                if text[start_index-1] in character:    # 단어의 일부로 매칭된 경우 (unit이 아닌 경우)
                    update=False
                elif text[start_index-1] in number: # 공백이나 숫자로 매칭되는 경우
                    pass # 공백이나 숫자로 매칭되는 경우 -> 그 자체로 unit 이거나, 숫자와 붙어있는 유닛인 경우
            elif (start_index == 0) and (end_index != len(text)): # 패턴 시작에 매칭되는 경우
                if text[end_index] in character:
                    update=False # 단어의 일부로 매칭된 경우로 볼 수 있다..
                elif text[end_index] in number:
                    pass # 끝단에 공백이나, 숫자가 등장하는 것은 유닛이 문자바로 앞에 등장하고 뒤에 공백이나 숫자가 바로 따라 붙는 경우를 의미한다.
            elif (start_index != 0) and (end_index != len(text)): #중간에 매칭된 경우
                if (text[start_index-1] in number) and (text[end_index] in number): # 둘다 숫자나 공백으로 둘러 쌓인 경우에만 유닛으로 볼 수 있다.
                    pass
                else:
                    if text[end_index] == "s" or text[end_index:end_index+2] == "es":
                        pass
                    else:
                        update=False
                # 다른 경우는 모두 문자의 일부로 매칭된 경우로 유닛으로 볼 수 없다.

            if update:
                unit = text[start_index:end_index]
                # reconstruct text
                text = text[:start_index] + " " + unit + text[end_index:]

                compared_alcohol['Measures'][i][j][0] = text
            vis[i].append(re.split(' ', text))

        else: #  in case that nothing matches
            vis[i].append(re.split(' ', text)) # 숫자만 존재하는 경우도 있으므로 넣도록 합니다.

# remove '' in the list
for i in range(len(vis)): # by row
    for j in range(len(vis[i])): # by ingredients
        remove_index = []
        for k in range(len(vis[i][j])): # by tokens
            if len(vis[i][j][k]) == 0:
                remove_index.append(k)

        for del_index in reversed(remove_index):
            a = vis[i][j].pop(del_index)

for i in range(len(vis)):
    remove_index = []
    for j in range(len(vis[i])):
        if (len(vis[i][j]) == 0):
            remove_index.append(j)
    for del_index in reversed(remove_index):
        vis[i].pop(del_index)

for i in range(len(vis)):
    print("index: ", i)
    print(vis[i])

index:  0
[['1', 'oz', 'white'], ['1', 'oz']]
index:  1
[['1/2', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/2', 'oz'], ['1/4', 'oz']]
index:  2
[['16', 'oz'], ['1.5', 'oz']]
index:  3
[['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz', 'bacardi'], ['1', 'oz'], ['1', 'oz'], ['3', 'oz'], ['1', 'oz'], ['1', 'cup']]
index:  4
[['1', 'shot'], ['2', 'shots'], ['1', 'shot'], ['1', 'shot']]
index:  5
[['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz', 'bacardi']]
index:  6
[['1/2', 'shot', 'bacardi'], ['1/2', 'shot']]
index:  7
[['1/3', 'oz'], ['1/3', 'oz'], ['1/3', 'oz']]
index:  8
[['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1-2', 'dash'], ['1', 'wedge']]
index:  9
[['2-3', 'oz']]
index:  10
[['2', '1/2', 'oz'], ['1', 'splash'], ['fill', 'with']]
index:  11
[]
index:  12
[['2', 'oz', 'dry'], ['4', 'oz'], ['0.75', 'oz']]
index:  13
[['1/3', 'part'], ['1/3', 'part'], ['1/3', 'part']]
index:  14
[['2', 'oz'], ['1/2', '

In [9]:
# 길이 검증

# for i in range(len(all_drinks)):
#     if len(vis[i]) != len(all_drinks['Measures'][i]):
#         print("different length!")
#         print("index: ", i)
#         print("len vis: ", len(vis[i]))
#         print("len all_drinks: ",len(all_drinks["Measures"][i]))
#         print()

In [10]:
# 위의 remove_duplicate_unit을 적용한 후로
# 단위는 모두 1개씩만 존재하게 된다. 그러나 단위가 애초에 없는 경우에도 'l', 'l' 이렇게 분류된 경우도 존재하기 때문에,
# 이 경우에도 대해서도 처리가 필요하다.
# 가능한 케이스는  -> 1. 'chilled' -> 'l', 'l', -> 'l'
#               -> 2. '0.5l' -> '0.5', 'l'
#               -> 3. 'l' -> 'l'
#               -> 4. '1.5 chilled' -> '1.5', 'l'
# 이중 숫자 단위, 단위만 존재한다면 이런 경우는 자동으로 해결된다.
# 문제는 관련 없는 문자만 등장하거나, 숫자 문자 이렇게 등장하는 경우이다.

def is_included(token, list_):
    if token in list_:
        return True
    if token + 's' in list_:
        return True
    if token + 'es' in list_:
        return True
    return False


# 구로 표현된 경우는 공백 분리로 인해 일치하는게 없을 것임으로 이런 것들은 따로 고려해줄 필요가 있다. 또한 구로 표현된 경우는 문자로 부터 분리된 경우를 걱정할 필요가 없다.
phrased = ["fl oz", "top up with", "fill to top", "to fill", "a little bit of", "juice of", "twist of", "long strip", "by taste", "to taste"]

for row in range(len(all_drinks)):
    list_ = all_drinks['Measures'][row]
    for i in range(len(list_)):
        element = list_[i][0]
        remove_index = [] # 각 element에 대해 삭제 index를 수집한다.
        for j in range(len(element)):
            tokens = element[j]
            if (tokens in UNIT) and (tokens not in phrased): # 각 행마다 유니크하게 존재하는 단위 (구로 표현된 단위는 제외)
                if is_included(tokens, vis[row][i]): # vis[row][each ingredient]
                    continue # -> 유닛의 단위가 존재하는 경우
                else:
                    remove_index.append(j) # -> 유닛의 단위가 존재하지 않아, 다른 단어로 부터 추출된 경우로 볼 수 있다.

        for del_index in reversed(remove_index):
            all_drinks['Measures'][row][i][0].pop(del_index)

#### Soultion 2

숫자 기반으로 매칭된 end_index 이후 부분은 포함시키지 않습니다.

단위로 매칭된 이후 부분은 단위가 아닌 문자에서 매칭된 것이 아니라면은 제거 대상이기 때문입니다.

In [11]:
number = ['1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '/']  # empty space or / for the case of number/unit differentiation
character = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

vis = []
for i in range(len(compared_alcohol)):
    vis.append([])
    for j in range(len(compared_alcohol['Measures'][i])):
        text = str(compared_alcohol['Measures'][i][j][0]).lower() # lower cased
        text = re.sub(',', '', text)
        text = re.sub('\\r|\\n', '', text)

        unit_index = re.search(filter, text) # index_start ~ index_end-1 까지가 포함되는 단위이다. 즉 [start_index : end_index] 이렇게 사용할 수 있다.

        update = True

        if unit_index: # in case of not NoneType -> matched at least one
            start_index, end_index = unit_index.start(), unit_index.end()
            if (start_index == 0) and (end_index == len(text)): # 단위만 나오는 경우
                pass
            elif (start_index != 0) and (end_index == len(text)): # 패턴에 끝에 매칭되는 경우
                if text[start_index-1] in character:    # 단어의 일부로 매칭된 경우 (unit이 아닌 경우)
                    update = False
                elif text[start_index-1] in number: # 공백이나 숫자로 매칭되는 경우
                    pass # 공백이나 숫자로 매칭되는 경우 -> 그 자체로 unit 이거나, 숫자와 붙어있는 유닛인 경우
            elif (start_index == 0) and (end_index != len(text)): # 패턴 시작에 매칭되는 경우
                if text[end_index] in character:
                    update = False # 단어의 일부로 매칭된 경우로 볼 수 있다..
                elif text[end_index] in number:
                    pass # 끝단에 공백이나, 숫자가 등장하는 것은 유닛이 문자바로 앞에 등장하고 뒤에 공백이나 숫자가 바로 따라 붙는 경우를 의미한다.
            elif (start_index != 0) and (end_index != len(text)): #중간에 매칭된 경우
                if (text[start_index-1] in number) and (text[end_index] in number): # 둘다 숫자나 공백으로 둘러 쌓인 경우에만 유닛으로 볼 수 있다.
                    pass
                else:
                    if (text[end_index] == "s") or (text[end_index:end_index+2] == "es"): # 복수형을 카운트하기 위한 경우
                        pass
                    else:
                        update = False
                # 다른 경우는 모두 문자의 일부로 매칭된 경우로 유닛으로 볼 수 없다.

            if update:
                unit = text[start_index:end_index]
                # reconstruct text
                text = text[:start_index] + " " + unit # text[end_index:] 이거는 필요가 없습니다.. 어차피 단위 이후 부분은 필요 없는 정보이기 때문입니다.

            vis[i].append(re.split(' ', text))

        else: #  in case that nothing matches
            vis[i].append(re.split(' ', text)) # 숫자만 존재하는 경우도 있으므로 넣도록 합니다.

# remove '' in the list
for i in range(len(vis)): # by row
    for j in range(len(vis[i])): # by ingredients
        remove_index = []
        for k in range(len(vis[i][j])): # by tokens
            if len(vis[i][j][k]) == 0:
                remove_index.append(k)

        for del_index in reversed(remove_index):
            a = vis[i][j].pop(del_index)

for i in range(len(vis)):
    remove_index = []
    for j in range(len(vis[i])):
        if (len(vis[i][j]) == 0):
            remove_index.append(j)
    for del_index in reversed(remove_index):
        vis[i].pop(del_index)

vis

[[['1', 'oz'], ['1', 'oz']],
 [['1/2', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/2', 'oz'],
  ['1/4', 'oz']],
 [['16', 'oz'], ['1.5', 'oz']],
 [['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1', 'oz'],
  ['1', 'oz'],
  ['3', 'oz'],
  ['1', 'oz'],
  ['1', 'cup']],
 [['1', 'shot'], ['2', 'shot'], ['1', 'shot'], ['1', 'shot']],
 [['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz']],
 [['1/2', 'shot'], ['1/2', 'shot']],
 [['1/3', 'oz'], ['1/3', 'oz'], ['1/3', 'oz']],
 [['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1-2', 'dash'],
  ['1', 'wedge']],
 [['2-3', 'oz']],
 [['2', '1/2', 'oz'], ['1', 'splash'], ['fill']],
 [],
 [['2', 'oz'], ['4', 'oz'], ['0.75', 'oz']],
 [['1/3', 'part'], ['1/3', 'part'], ['1/3', 'part']],
 [['2', 'oz'], ['1/2', 'oz'], ['1', 'splash'], ['1', 'oz']],
 [['1', 'oz'], ['1/2', 'oz'], ['4', 'oz'], ['1/2', 'oz']],
 [['2', 'oz'], ['4', 'oz'], ['1', 'twist', 'of']

In [12]:
phrased = ["fl oz", "top up with", "fill to top", "to fill", "a little bit of", "juice of", "twist of", "long strip", "by taste", "to taste"]

def is_num_included(str):
    number = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for num in number:
        if num in str:
            return True
    return False

# combine phrased representation
start_phrase = ["fl", "top", "fill", "to", "a", "juice", "twist", "long", "by", "to"]
for i in range(len(vis)):
    for j in range(len(vis[i])):
        index = len(vis[i][j])
        for k in range(len(vis[i][j])):
            if vis[i][j][k] in start_phrase:
                index = k
                break
        st = ""
        for combine in range(index, len(vis[i][j])):
            if combine == index:
                st += vis[i][j][combine]
            else:
                st += " " + vis[i][j][combine]
        for del_index in range(len(vis[i][j])-1, index-1, -1):
            vis[i][j].pop(del_index)

        if len(st) != 0:
            vis[i][j].append(st)

        remove_index = []
        for k in range(len(vis[i][j])):
            if (vis[i][j][k] not in UNIT) and not(is_num_included(vis[i][j][k])):
                remove_index.append(k)
        for del_index in reversed(remove_index):
            vis[i][j].pop(del_index)

In [13]:
for i in range(len(vis)):
    if len(vis[i]) != len(all_drinks['Measures'][i]):
        print("different length!")
        print("index: ", i)
        print("len vis: ", len(vis[i]))
        print("len all_drinks: ",len(all_drinks["Measures"][i]))

In [14]:
# 솔루션 검증

for i in range(len(all_drinks)):
    for j in range(len(all_drinks["Measures"][i])):
        if (all_drinks["Measures"][i][j][0] != vis[i][j]):
            print("index: ", i)
            print("all_drinks: ", all_drinks["Measures"][i][j][0])
            print("vis: ", vis[i][j])
            print("compared_alcohol: ", compared_alcohol["Measures"][i][j][0])
            print()


index:  8
all_drinks:  ['1', '2']
vis:  ['1-2']
compared_alcohol:  1-2 dash 

index:  9
all_drinks:  ['2', '3', 'oz']
vis:  ['2-3', 'oz']
compared_alcohol:  2-3  oz

index:  21
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  29
all_drinks:  ['juice of', '1/4']
vis:  ['juice of']
compared_alcohol:   juice of 1/4 

index:  45
all_drinks:  ['1', '3', 'pint']
vis:  ['1-3', 'pint']
compared_alcohol:  1-3  pint 

index:  54
all_drinks:  ['juice of', '1']
vis:  ['juice of']
compared_alcohol:   juice of 1 

index:  72
all_drinks:  ['3', '4', 'cup']
vis:  ['3-4', 'cup']
compared_alcohol:  3-4  cups 

index:  88
all_drinks:  ['3', '6']
vis:  ['3-6']
compared_alcohol:  3-6 crushed 

index:  126
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  148
all_drinks:  ['2', '3', 'drop']
vis:  ['2-3', 'drop']
compared_alcohol:  2-3  drops 

index:  156
all_drinks:  ['4']
vis:  ['4', 'l']
compared_alcohol:  4 tab le

In [15]:
all_drinks.to_pickle('./preprocessed_alcohol_v2.pkl')